In [86]:
import pandas as pd

# Wczytaj plik CSV z dysku Google Drive
books = pd.read_csv("D:/Downloads/archive(4)/Books.csv",low_memory=False)
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [76]:
print(books[books['Book-Title'].str.startswith('Harry')][['Book-Title', 'ISBN']])
print(books['ISBN']=="059035342X")

                                               Book-Title        ISBN
2143    Harry Potter and the Sorcerer's Stone (Harry P...  059035342X
2809       Harry Potter and the Sorcerer's Stone (Book 1)  0590353403
3459     Harry Potter and the Chamber of Secrets (Book 2)  0439064872
3839    Harry Potter and the Prisoner of Azkaban (Book 3)  0439136350
5431         Harry Potter and the Goblet of Fire (Book 4)  0439139597
...                                                   ...         ...
257263  Harry Potter and the Goblet of Fire (Harry Pot...  043955490X
259611              Harry Potter and the Sorcerer's Stone  0939173344
260025             Harry Goes to Camp (Monsterkids, No 3)  0061062383
265961       Harry and Catherine (Vintage Contemporaries)  0679730761
266651             Harry Houdini: Escape Artist (Level 2)  0689848153

[144 rows x 2 columns]
0         False
1         False
2         False
3         False
4         False
          ...  
271355    False
271356    False
271357  

In [129]:
books=books[['ISBN','Book-Title']]


books.isnull().sum()
books.drop_duplicates()


,ISBN,Book-Title
0,0195153448,Classical Mythology
1,0002005018,Clara Callan
2,0060973129,Decision in Normandy
3,0374157065,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi
...,...,...
271355,0440400988,There's a Bat in Bunk Five
271356,0525447644,From One to One Hundred
271357,006008667X,Lily Dale : The True Story of the Town that Ta...
271358,0192126040,Republic (World's Classics)


# Zabawa z Gradio

In [5]:
import gradio as gr

def greet(name):
    return get_recomendations(name,1).tolist()

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
    
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Tutaj zaczyna się coś ciekawego

In [130]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy.sparse import csr_matrix

# Zakładając, że books to DataFrame z kolumną 'Book-Title'
vectorizer_model = CountVectorizer()
wektory_cech = vectorizer_model.fit_transform(books['Book-Title'])

# Przekształć na rzadką macierz
wektory_cech_rzadkie = csr_matrix(wektory_cech)

In [131]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import pairwise_distances

def closest_elements_sparse(target_title, n=5):
    # Przekształć tytuł docelowy do wektora cech
    target_vector = vectorizer_model.transform([target_title])

    # Oblicz odległość euklidesową między wektorem docelowym a wszystkimi wektorami cech
    distances = pairwise_distances(target_vector, wektory_cech_rzadkie, metric="l2", n_jobs=-1)

    # Znajdź indeksy najbliższych elementów
    closest_indices = distances.argsort()[0][:n]

    # Pobierz najbliższe elementy
    closest_elements_list = books.iloc[closest_indices]['Book-Title'].tolist()

    return closest_elements_list

In [132]:
closest_elements_sparse("The Lord of The Rings: Two Towers",5)

['The Two Towers (Lord of the Rings (Hardcover))',
 'The Two Towers (Lord of the Rings (Paperback))',
 'The Two Towers (Lord of the Rings Part 2)',
 'The Lord of the Rings',
 'The Lord of the Rings']

## Szkoda mi bylo to usuwac na razie

In [27]:
from Levenshtein import distance
def closest_title(title):
    # Utwórz DataFrame z indeksem
    m = pd.DataFrame(books.index)
    m['lev'] = m.index.map(lambda x: distance(books['Book-Title'].iloc[x], title))

    # Znajdź najbliższy tytuł na podstawie odległości Levenshteina
    closest_title = books.loc[m.sort_values('lev', ascending=True).index[0], 'Book-Title']

    return closest_title

def n_najmniejszych(lista, n):
    # Utwórz listę indeksów dla elementów różnych od zera
    indeksy_niezerowych_elementow = [indeks for indeks, wartosc in enumerate(lista[0]) if abs(wartosc) > 1e-10]
    
    # Posortuj listę indeksów według wartości
    posortowane_indeksy = sorted(indeksy_niezerowych_elementow, key=lambda x: lista[0][x])
    
    # Ogranicz się do pierwszych n najmniejszych indeksów
    n_najmniejsze_indeksy = posortowane_indeksy[:n]
    
    return n_najmniejsze_indeksy


def get_recomendation(title,n):
    check_book = any(books['Book-Title'].str.contains(title, case=False))
    if check_book == True:
        print("1")
        row_number = books.index[books['Book-Title'].str.contains(title, case=False)].tolist()
        row_number =row_number[0]
        array = dane[row_number][1].tolist()
        title_list=n_najmniejszych(array,n)
        tytul_ksiazki =[]
        for indeks in title_list:
            tytul_ksiazki.append(books["Book-Title"].iloc[indeks])

    else:
        print("2")
        title = closest_title(title)
        row_number = books.index[books['Book-Title'].str.contains(title, case=False)].tolist()
        row_number =row_number[0]
        array = dane[row_number][1].tolist()
        title_list=n_najmniejszych(array,n)
        tytul_ksiazki =[]
        for indeks in title_list:
            tytul_ksiazki.append(books["Book-Title"].iloc[indeks])
            
    return(tytul_ksiazki)

get_recomendation("Dark Tales",4)
#get_recomendation("The Hobbit : The Enchanting Prelude to The Lord of the Rings",4)



1


IndexError: list index out of range

# Rekomendaja po Popularności danej książki z uzycie podobienstwa cosinusowego

In [3]:
#wczytanie danych z ocenami
ratings = pd.read_csv("D:/Downloads/archive(4)/Ratings.csv",low_memory=False)
ratings.head()


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
#połącz danych 
book_rating = pd.merge(books,ratings, on ='ISBN')
book_rating.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [6]:
# Liczba ocen dla każdej książki
num_book_rating = book_rating.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_book_rating.rename(columns = {'Book-Rating':'num_rating'}, inplace=True)
num_book_rating.head()

,Book-Title,num_rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [7]:
# Średnia ocena dla każdej książki
avg_book_rating = book_rating.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_book_rating.rename(columns={'Book-Rating':'avg_rating'}, inplace=True)
avg_book_rating.head()

C:\Users\13luk\AppData\Local\Temp\ipykernel_17984\339634649.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_book_rating = book_rating.groupby('Book-Title').mean()['Book-Rating'].reset_index()


,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00


In [8]:
#połączenie danych sredniej oceny i liczby ocen
books_df = avg_book_rating.merge(num_book_rating, on = 'Book-Title')
books_df.head()

,Book-Title,avg_rating,num_rating
0,A Light in the Storm: The Civil War Diary of ...,2.25,4
1,Always Have Popsicles,0.00,1
2,Apple Magic (The Collector's series),0.00,1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00,1
4,Beyond IBM: Leadership Marketing and Finance ...,0.00,1


In [12]:
#filtrowanie uzytkownikow powyzej ktorzy maja wiecej niz 200 ocen
usr = book_rating.groupby("User-ID").count()["Book-Title"] > 200 

read_book = usr[usr].index
filtered_rating = book_rating[book_rating["User-ID"].isin(read_book)]

In [17]:
#filtrowanie ksiazek, ktore maja wiecej niz 50 ocen
famous = filtered_rating.groupby("Book-Title").count()["Book-Rating"] >= 50
famous_book = famous[famous].index
final_ratings = filtered_rating[filtered_rating["Book-Title"].isin(famous_book)]

In [18]:
#tabela przestawna wiersz-tytul, kolumna-uzytkownik, wartosc-ocena
pt = final_ratings.pivot_table(index="Book-Title", columns="User-ID", values="Book-Rating")
pt.fillna(0, inplace=True)

In [19]:
#model podobienstwa cosinusowego
from sklearn.metrics.pairwise import cosine_similarity
similarity_score = cosine_similarity(pt)

In [23]:
#funkcja zwracajaca podobne ksiazki
import numpy as np

def recommend_books(book_name, n=5):
    # Znajdź indeks książki w tabeli przestawnej
    index = np.where(pt.index == book_name)[0][0]
    
    # Posortuj podobieństwa i wybierz top n
    similar_books = sorted(enumerate(similarity_score[index]), key=lambda x: x[1], reverse=True)[1:n+1]

    # Zbierz tylko tytuły książek
    recommended_titles = []

    for bk in similar_books:
        temp_df = books[books["Book-Title"] == pt.index[bk[0]]]
        recommended_titles.append(temp_df.drop_duplicates("Book-Title")["Book-Title"].values[0])

    return recommended_titles

In [24]:
recommend_books("1st to Die: A Novel")

['Along Came a Spider (Alex Cross Novels)',
 'Roses Are Red (Alex Cross Novels)',
 'Pop Goes the Weasel',
 'Violets Are Blue',
 'Lightning']

## Rekomendacja po odleglosci Euklidesowej

In [124]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy.sparse import csr_matrix

# Zakładając, że books to DataFrame z kolumną 'Book-Title'
vectorizer_model = CountVectorizer()
wektory_cech = vectorizer_model.fit_transform(books['Book-Title'])

# Przekształć na rzadką macierz
wektory_cech_rzadkie = csr_matrix(wektory_cech)

In [125]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import pairwise_distances

def closest_elements_sparse(target_title, n=5):
    # Przekształć tytuł docelowy do wektora cech
    target_vector = vectorizer_model.transform([target_title])

    # Oblicz odległość euklidesową między wektorem docelowym a wszystkimi wektorami cech
    distances = pairwise_distances(target_vector, wektory_cech_rzadkie, metric="l2", n_jobs=-1)

    # Znajdź indeksy najbliższych elementów
    closest_indices = distances.argsort()[0][:n]

    # Pobierz najbliższe elementy
    closest_elements_list = books.iloc[closest_indices]['Book-Title'].tolist()

    return closest_elements_list

In [126]:
closest_elements_sparse("The Lord of The Rings: Two Towers",5)

['The Two Towers (Lord of the Rings (Hardcover))',
 'The Two Towers (Lord of the Rings (Paperback))',
 'The Two Towers (Lord of the Rings Part 2)',
 'The Lord of the Rings',
 'The Lord of the Rings']